In [1]:
#Since the official Twitter API doesn't allow to get tweets older than 1 week, I'm using GetOldTweets to parse historical data. 
#Check @noahberhe's code for more details: https://github.com/noahberhe/COVID-19/blob/master/COVID-19%20Winners%20and%20Losers.ipynb 

import GetOldTweets3 as got
from tqdm.notebook import tqdm
import sys
import time
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib import animation
import mpl_toolkits.mplot3d.art3d as art3d
import ipywidgets as widgets
# interact_manual
from IPython.display import display

from ipywidgets import *
import seaborn as sns
from datetime import date, timedelta, datetime
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import make_pipeline
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.power import tt_ind_solve_power, TTestIndPower
import pylab

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacy
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
from wordcloud import WordCloud, STOPWORDS
import re

pd.set_option('display.min_rows', 100, 'display.max_colwidth', 500)

In [2]:
def getoldtweets(since, until, query, maxtweets=10000):
    '''Wrap TweetCriteria class in a function to enable dynamic setting of variables when called'''
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch('{}'.format(query)) \
                                           .setSince('{}'.format(since)) \
                                           .setUntil('{}'.format(until)) \
                                           .setMaxTweets(maxtweets)
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    return tweets

In [5]:
start = '2020-07-18'
until = (datetime.today()-timedelta(days=1)).date()
datelist = pd.date_range(start, end=until).tolist()
query = 'belarus OR беларусь OR minsk OR минск OR lukashenko OR лукашенко OR лукашэнка OR babariko OR бабарико OR babaryka OR бабарыка OR тихановский OR тихановская OR цепкало OR tikhanovskaya OR tikhanovskiy OR tsepkalo OR tsepkala'
sleep_mins = 16
max_attempts = 3

tweets_dict = {}
error = None
for (i,date) in tqdm(enumerate(datelist)):
    attempts_at_date = 0
    if error != 'KeyboardInterrupt':
        if attempts_at_date < max_attempts:
            while True:
                if attempts_at_date < max_attempts:
                    attempts_at_date += 1
                    try:
                        print('\nAttempt {} of {} retrieving {}'\
                              .format(attempts_at_date, max_attempts, str(date)[:10]))
                        tweets_dict[str(date)[:10]] = getoldtweets(str(date)[:10],
                                                                   str(date+timedelta(days=1))[:10],
                                                                   query)
                        num_tweets = len(tweets_dict[str(date)[:10]])
                        print('Success retrieving {} tweets for {}: {} of {} dates'\
                              .format(num_tweets,str(date)[:10],i+1,len(datelist)))
                        break

                    except KeyboardInterrupt:
                        error = 'KeyboardInterrupt'
                        break

                    except:
                        try:
                            print('\nError retrieving {} on attempt {} of {}. Sleeping for {} minutes.\nMinutes Slept:'\
                                  .format(str(date)[:10], attempts_at_date, max_attempts, sleep_mins+attempts_at_date))
                            for t in range(sleep_mins+attempts_at_date):
                                sys.stdout.write(str(t) + '.. ')
                                sys.stdout.flush()
                                time.sleep(60)
                        except KeyboardInterrupt:
                            break
                else:
                    print('Attempt {} at {} failed. Exiting.'.format(max_attempts, str(date)[:10]))
                    break
        else:
            break
    else:
        break

tweets_ls = []
for tweetquery in list(tweets_dict.values()):
    for tweet in tweetquery:
        tweets_ls.append(tweet)

print('\nTotal No. Tweets retrieved: {}'.format(len(tweets_ls)))


Attempt 1 of 3 retrieving 2020-07-18
Success retrieving 3391 tweets for 2020-07-18: 1 of 9 dates

Attempt 1 of 3 retrieving 2020-07-19
Success retrieving 4443 tweets for 2020-07-19: 2 of 9 dates

Attempt 1 of 3 retrieving 2020-07-20
Success retrieving 4613 tweets for 2020-07-20: 3 of 9 dates

Attempt 1 of 3 retrieving 2020-07-21
Success retrieving 4463 tweets for 2020-07-21: 4 of 9 dates

Attempt 1 of 3 retrieving 2020-07-22
Success retrieving 4704 tweets for 2020-07-22: 5 of 9 dates

Attempt 1 of 3 retrieving 2020-07-23
Success retrieving 4591 tweets for 2020-07-23: 6 of 9 dates

Attempt 1 of 3 retrieving 2020-07-24
Success retrieving 5039 tweets for 2020-07-24: 7 of 9 dates

Attempt 1 of 3 retrieving 2020-07-25
Success retrieving 3555 tweets for 2020-07-25: 8 of 9 dates

Attempt 1 of 3 retrieving 2020-07-26
Success retrieving 3791 tweets for 2020-07-26: 9 of 9 dates


Total No. Tweets retrieved: 38590


In [6]:
def spacy_text_processor(text_col, pos=['PROPN']):
    '''Clean punctuation and tokenize tweets. Filter tokens by token.pos_ specified. Entity Recognition'''
    
    nlp = spacy.load('en_core_web_sm')

    persons = [('alexander lukashenko','lukashenko'),('viktor','babariko'),('babaryka','babariko'),('president of belarus','lukashenko'), ('лукашенко', 'lukashenko'), ('лукашэнка', 'lukashenko'),
               ('babaryko','babariko'),('alexandr lukashenko','lukashenko'),('lukashenko','lukashenko'),('babariko','babariko'), ('бабарико','babariko'), ('бабарыка', 'babariko'), ('tsepkalo','tsepkalo'),('tsepkala','tsepkalo'),('tsapkala','tsepkalo'),('цепкало', 'tsepkalo'), ('цэпкала', 'tsepkalo'), ('цапкала', 'tsepkalo'), 
               ('cichanouskaja','tikhanovskaya'),('tikhanovskaya','tikhanovskaya'),('tikhanovskaya','tikhanovskaya'),('tsikhanouskaya', 'tikhanovskaya'), ('тихановская','tikhanovskaya'), ('тихановский','tikhanovskaya'), ('ціханоўская','tikhanovskaya'),
               ('babarik','babariko'),('lukashenka','lukashenko')]
    orgs = [('opposition','opposition'),('оппозиция','opposition'), ('government','government'),('правительство','government'),('eu','eu'),('european union','eu'), ('us','us'), ('usa','us'), ('u.s.a.','us'), ('u.s.','us'), ('america','us')]
    
    tokens = []
    people = []
    organisations = []
    custom_people = []
    custom_organisations = []

    for parsed in tqdm(nlp.pipe(text_col)):
        doc_ = ''
        for token in parsed:
            if token.pos_ in pos:
                doc_ += str(token) + ' '
            else:
                pass
        doc_ = doc_.strip()

        tokens.append(doc_)

        if parsed.ents:
            for ent in parsed.ents:
                named_persons = ''
                named_orgs = ''
                if ent.label_ == 'PERSON':
                    named_persons += ent.text + ''
                if ent.label_ == 'ORG':
                    named_orgs += ent.text + ''
                else:
                    pass
        else:
            named_persons = ''
            named_orgs = ''
        
        people.append(named_persons)
        organisations.append(named_orgs)
        
    ruler_people = EntityRuler(nlp, overwrite_ents=True)
    ruler_orgs = EntityRuler(nlp, overwrite_ents=True)

    for (p,i) in persons:
        ruler_people.add_patterns([{"label": "PERSON", "pattern": [{"LOWER": p}], "id": i}])
    for (o,i) in orgs:
        ruler_orgs.add_patterns([{"label": "ORG", "pattern": [{"LOWER": o}], 'id': i}])

    ruler = EntityRuler(nlp)
    ruler.add_patterns(ruler_orgs.patterns)
    ruler.add_patterns(ruler_people.patterns)

    nlp.add_pipe(ruler)

    for parsed in tqdm(nlp.pipe(text_col, disable=['ner'])):
        if parsed.ents:
            for ent in parsed.ents:
                custom_persons = ''
                custom_orgs = ''
                if ent.label_ == 'PERSON':
                    custom_persons += ent.ent_id_ + ''
                if ent.label_ == 'ORG':
                    custom_orgs += ent.ent_id_ + ''
                else:
                    pass
        else:
            custom_persons = ''
            custom_orgs = ''

        custom_persons = custom_persons.strip()
        custom_orgs = custom_orgs.strip()

        custom_people.append(custom_persons)
        custom_organisations.append(custom_orgs)
        
    return tokens, people, organisations, custom_people, custom_organisations

In [7]:
def vader_scorer(df):
    '''Compute vaderSentiment scores for each tweet
    Args: Dataframe containing a 'text' column
    Returns: Dataframe of vader scores
    '''
    analyzer = SentimentIntensityAnalyzer()
    vader_scores = df.loc[:,'text'].map(analyzer.polarity_scores)

    dvec = DictVectorizer()
    vader_scores = dvec.fit_transform(vader_scores)
    
    vader_scores = pd.DataFrame(vader_scores.toarray(),columns=dvec.feature_names_)
    return vader_scores

In [8]:
def tweet_postproc_tocsv(tweets_dictionary):
    '''Applying NLP post-processing to newly retrieved tweets, and save to csv file, one for each date queried
    Args: a dictionary of tweet objects, each key pertaining to each date queried for
    Returns: nothing. Output is saved in a directory of tweet .csv files for import during analysis
    '''
    for query_date in list(tweets_dictionary.keys()):
        tweets_df = []
        for tweet in tweets_dictionary[query_date]:
            tweets_df.append([tweet.username, tweet.id, tweet.date, tweet.favorites, tweet.hashtags,
                              tweet.mentions, tweet.retweets, tweet.replies, tweet.text])

        tweets_df = pd.DataFrame(tweets_df,columns=['username','tweet_id','date','favorites','hashtags',
                                                    'mentions','retweets','replies','text'])

        propn_string, people, orgs, topical_persons, topical_orgs = spacy_text_processor(tweets_df['text'])

        pd.options.mode.chained_assignment = None

        tweets_df['topical_persons'] = topical_persons
        tweets_df['topical_orgs'] = topical_orgs
        tweets_df['persons'] = people
        tweets_df['orgs'] = orgs
        tweets_df['propn'] = propn_string

        vader_scores = vader_scorer(tweets_df)
        tweets_df = pd.concat([tweets_df,vader_scores], axis=1)

        tweets_df = tweets_df.drop_duplicates(subset='tweet_id', ignore_index=True)
        tweets_df.to_csv('tweets_{}.csv'.format(query_date), index=False)

In [9]:
def csv_compiler(folder='tweets_*.csv'):
    '''Import files as specified in input and collate into one DataFrame
    Args: folder, str. filepath relative to current folder, with optional pattern recognition
    Returns: DataFrame of collated data
    '''
    df = pd.concat([pd.read_csv('{}'.format(filepath), parse_dates=['date']) for filepath in glob.iglob(folder)],
                   ignore_index=True)
    df.sort_values("date", inplace=True)
    df.drop_duplicates(subset="tweet_id", ignore_index=True, inplace=True)
    return df

def emotive_filter(corpus):
    text_to_list = re.sub("[^\w]", " ",  corpus).split()
    list_to_df = pd.DataFrame(text_to_list, columns=['text'])
    return list_to_df, vader_scorer(list_to_df)

def labeller(sentiment_col, abs_threshold=0.05):
    if sentiment_col < -abs_threshold:
        return 'NEG'
    elif sentiment_col > abs_threshold:
        return 'POS'
    else:
        return 'NEU'

In [10]:
tweet_postproc_tocsv(tweets_dict)

/opt/anaconda3/envs/daria_allwomen/lib/python3.7/site-packages/spacy/util.py:271: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.0). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [11]:
tweets = csv_compiler()

In [12]:
tweets.to_csv('last_dataset.csv')

In [18]:
df_lukashenko = df[df.topical_persons == 'lukashenko']

In [19]:
df_lukashenko.shape

(8023, 19)

In [20]:
df_babariko = df[df.topical_persons == 'babariko']

In [21]:
df_babariko.shape

(684, 19)

In [22]:
df_tsepkalo = df[df.topical_persons == 'tsepkalo']

In [23]:
df_tsepkalo.shape

(1024, 19)

In [24]:
df_tikhanovskaya = df[df.topical_persons == 'tikhanovskaya']

In [25]:
df_tikhanovskaya.shape

(1015, 19)

In [26]:
import json
from apiclient.discovery import build
import os
import time

pd.options.mode.chained_assignment = None

In [27]:
import requests

In [28]:
import socket
from googleapiclient import discovery

In [29]:
from google.api_core import retry

In [31]:
def translation_googleAPI(query, target_language):

    socket.setdefaulttimeout(600)  # set timeout to 10 minutes
    service = discovery.build('translate','v2',developerKey='INSERT YOUR OWN KEY')
    collection = service.translations() 
    request = collection.list(q=query, target=target_language) 
    response = request.execute() 
    response_json = json.dumps(response) 
    translation = ((response['translations'][0])['translatedText'])
    print (translation)
    print('....')
    return translation

In [32]:
df_tikhanovskaya['translated'] = df_tikhanovskaya['text'].apply(lambda x: translation_googleAPI(x,'en'))

BBC News Russian Service - Elections in Belarus: Presidential Candidate Svetlana Tikhanovskaya Demands an End to Abuse of Arrested Husband
....
Svetlana Tikhonovskaya: I&#39;m moving on Svetlana Tikhanovskaya: I go further https://youtu.be/zYTykBK1T8o via @YouTube
....
BBC News Russian Service - Elections in Belarus: Presidential Candidate Svetlana Tikhanovskaya Demands an End to Abuse of Arrested Husband
....
So I&#39;m wondering if Tikhanovskaya does take part in the elections, then the cockroach needs to be falsified on a huge scale, will he go for it or get pissed off?
....
Our opposition lacks competent PR specialists. They are good fellows all that got out. Tikhanovsky immediately raised a fuss .. But now specific phrases are needed. Specific videos. With examples as for morons, so that even grandmothers get it. Because believe me! They don&#39;t get it! They are fine.
....
Svetlana Tikhanovskaya icon of modern feminism
....
#CUB The Belarusian CEC did not allow two opposition pr

https://youtu.be/r60fv0mmtvw Lukashenko Belarus Minsk Tsepkalo Babariko Tikhanovskaya
....
I&#39;m going to the Legal rally in support of #Tikhanovskaya I will make a thread about all this.
....
&quot;The authorities got scared, repression began.&quot; Tikhanovskaya spoke at a picket in Dzerzhinsk
....
People are starting to gather in Minsk for presidential candidate Tsikhanouskaya's picket. The elections are in three week's time #Belarus 
....
Analyst: after becoming president, Tikhanovskaya will face a huge number of serious political and economic problems on the very first day | http://EX-PRESS.BY: https://ex-press.by/rubrics/politika/2020/07/19/analitik-stav-prezidentom-tixanovskaya-v-pervyj-zhe-den-stolknetsya-s- ogromnym-kolichestvom-sereznyx-politicheskix-i-ekonomicheskix-problem? utm_referrer = https% 3A% 2F% 2Fzen.yandex.com
....
&quot;The authorities got scared, repression began.&quot; Tikhanovskaya spoke at a picket in Dzerzhinsk
....
The presidential candidate of Belarus Sv

 : Enormous rally in #Minsk today in support of opposition candidate Sviatlana Tsikhanouskaya. The pro-democracy movement continues in #Belarus. 
....
In the area of Bangalore square Svetlana Tikhanovskaya held a rally. https://youtu.be/dhV1HaSv4ZQ via @YouTube
....
The Belarusian people will rise to defend their votes in the event of election fraud. This opinion was expressed by the presidential candidate Svetlana Tikhanovskaya on July 19 at her first election rally in Minsk in the Friendship of Peoples Park. # to go out # to the dictators
....
now the only candidate is Tikhanovskaya, after the victory she leaves and calls real elections
....
The only opposition candidate is Tikhanovskaya. And after the elections, Sashka 3% admit that it was he who flunked Olof Palme, Ceausescu and Kennedy.
....
With these elections, I began to have political dreams. For example, today I dreamed that Tikhanovskaya got 51% of the votes, and the Current President 49% of the votes, and I was so happy
...

Svetlana Tikhanovskaya sent children outside Belarus
....
“In a safe place” - presidential candidate Tikhanovskaya took children out of Belarus https://telegraf.by/politika/v-bezopasnom-meste-kandidat-v-prezidenty-tihanovskaya-vyvezla-detej-iz-belarusi/
....
Following earlier threats to her family, presidential candidate Tsikhanouskaya has moved her children outside Belarus #electby
....
Svetlana Tikhanovskaya took her children out of Belarus
....
Selvestor! Yesterday in Minsk, Tikhanovskaya and some joint headquarters of quilted jackets also called for white ribbons to be tied? Probably wait for the grain to bless !!!
....
Svetlana Tikhanovskaya took the children out of the country: They are in a safe place Children of the candidate in ... https://belaruspartisan.by/politic/506661/?utm_source=twitter&amp;utm_medium=cpc
....
I think if you click &quot;show more&quot;, then there will be Tikhanovsky, and this whole gang of watering cans
....
Forced distribution, Ravkov&#39;s law - What 

The candidate for the presidency of Belarus Svetlana Tikhanovskaya took the children out of the country to their homeland.
....
The presidential candidate of Belarus took her children away from the country Earlier, Svetlana Tikhanovskaya reported about threats against her children if she did not refuse to participate in the elections
....
The presidential candidate of Belarus Svetlana Tikhanovskaya took the children to the European Union, as they were threatened against the background of the election race. Four-year-old daughter and ten-year-old son left the country and are with their grandmother
....
The CEC published information on applications for pickets from candidates. Alexander Lukashenko - 884 public events. Andrey Dmitriev - 214. Svetlana Tikhanovskaya - 160. Sergei Cherecheny - 138. Anna Kanopatskaya - 0. One of the candidates submitted his IQ test instead of applications.
....
hmm, well, let&#39;s say &quot;ladies from Victor Babariko&#39;s headquarters and candidate Svetlan

...

“President Lukashenka […] has ruled Belarus for 26 years. In this time, no election has been deemed free or fair by western goverments or observers.” Svyatlana Tsikhanouskaya, unica candidata rimasta, vuole vincere per dare un'elezione davvero democratica 
....
here is zhezh Lukashenka murzilka this Tikhanovskaya
....
LUKASHENKO WILL NOT BE PENSION. TIKHANOVSKAYA ASSOCIATES BELARUS. https://youtu.be/CK9gWSJAWlw via @YouTube
....
Svetlana Tikhanovskaya has three more rallies today. The first one is in Orsha, Lukashenko's homeland which he recently turned into a tax haven to support its collapsing economy. Didn't help, Tikhanovskaya still attracted massive crowds 
....
In Orsha, Tikhanovskaya gathered more than a thousand people at a rally and turned to the military http://dlvr.it/RcL9pZ #belarus
....
Svetlana Tikhanovskaya began her performance in Orsha in Belarusian VIDEO https://nn.by/?c=ar&amp;i=255930&amp;lang=ru
....
 𝗢𝗻 𝗝𝘂𝗹𝘆 𝟭𝟰 CEC (Central Election Commission) officially register

And Tikhanovsky? The Russian world will be delighted.
....
Presidential candidate Sviatlana Tsikhanouskaya continues to attract hundreds at her rallies across #Belarus Babruisk now. Pic @charter_97
....
Tikhanovskaya is something!
....
#Belarus. Yet another massive rally in #Babruysk, a city in the Mogilev region. It's the third rally of #Tsikhanouskaya today.Previous meetings in #Mogilev and #Orsha gathered thousands again. I don't remember such large pre-election rallies taking place all across the country
....
#Belarus Opposition candidate Tsikhanouskaya drew huge crowds in #Babruysk at her last election rally of the day. Earlier she was in #Orsha and #Mogilev where thousands of people showed up to support her
....
It feels like Korzh came to Bobruisk, not Tikhanovskaya
....
The best rally of presidential candidate Tsikhanouskaya so far taking place right now in Babruysk. Fantastic reception by full-stadium crowd. Feels like American presidential campaign rally Never seen something 

Tikhanovskaya in Bobruisk - like football in Brest: top atmosphere and the coolest &quot;starry sky&quot;
....
Tikhanovsky Sergey Leonidovich - http://Myrotvorets.center
....
Big crowd today in support of Sviatlana Tsikhanouskaya in Zhlobin, Belarus https://belarus.liveuamap.com/en/2020/26-july-big-crowd-today-in-support-of-sviatlana-tsikhanouskaya via @belamova #Belarus
....
Incredible emotions: Svetlana Tikhanovskaya and her team held a rally in Zhlobin (Online, video) - Charter&#39;97 :: News from Belarus - Belarusian News - News from Belarus - Republic of Belarus - Minsk http://0s.mnugc4tumvzdsnzon5zgo.nblz.ru/ru/news / 2020/7/26/387103 /
....
Tikhanovskaya is the Kremlin candidate. She may not even open her mouth. The layout is visible to the naked eye. Well, and her Komsomol speech with emotional anguish just causes laughter. The Kremlin has a stranglehold on the BSSR. Will not let go
....
#Belarus. If up to 3,000 came to support #Tsikhanouskaya in #Zhlobin, a city with a populat

The leader of the Belarusian protest movement Sergei Tikhanovsky illegally visited Crimea: https://informnapalm.org/48998-lider-belorusskogo-protestnogo-dvizh/
....
The leader of the Belarusian protest movement, Sergei Tikhanovsky, illegally visited Crimea:
....
The leader of the Belarusian protest movement, Sergei Tikhanovsky, illegally visited Crimea:
....
Svetlana Tikhanovskaya largest rally to date is happening right now in Gomel. Second largest city in Belarus and the home of her imprisoned husband Sergei 
....
Tikhanovskaya brought extras from Borisov in her jeeps.
....
#russianfuture why Tikhanovsky was able in Belarus, but @navalny traveling through the cities of Russia could not?
....
Arrested Belarusian oppositionist Tikhanovsky ended up in the Ukrainian &quot;Peacemaker&quot; - News at http://KP.UA
....
#Belarus Opposition candidate Tsikhanouskaya's rally in #Gomel. Possibly her largest one yet with thousands attending. Tsikhanouskaya's campaign has been very impressive: she

My bet is Lukashenko 79%, Tikhanovskaya 5-6%
....
Rather, just a sign is a lawyer, and a Tikhanovskaya philologist. In fact, new elections are possible and there are no obstacles
....
Lukashenko&#39;s rival gathered about 10 thousand supporters for a picket in Gomel http://www.rosbalt.ru/world/2020/07/27/1855627.html # Belarus # elections # Tikhanovskaya
....
Tikhanovskaya FOREIGN POLICY &quot;We must create a partnership on an equal footing, and not dependence on gas or credit&quot; needles. &quot;There is no simple way. Either we unite and move on to the country for life together, or we repeat the mistakes of past years again.&quot; Who are we teaming up with? It&#39;s hot.
....
I went to social networks and went crazy, degenerates who in 2014 were thrown out of the social circle for reasons of information hygiene (cannibals-warmongers), today turned out to be those who admire the stand-ups of a bald restaurateur, and hang tags I / we / Tikhanovskaya. I spit you on the cured meat.
..

In [33]:
df_tikhanovskaya.head(20)

,Unnamed: 0,username,tweet_id,date,favorites,hashtags,mentions,retweets,replies,text,topical_persons,topical_orgs,persons,orgs,propn,compound,neg,neu,pos,translated
53,53,DJ_ALEXX57,1284293958789730304,2020-07-18 01:08:35+00:00,1,NaN,NaN,0,0,BBC News Русская служба - Выборы в Беларуси: кандидат в президенты Светлана Тихановская требует прекратить издевательства над арестованным мужем,tikhanovskaya,NaN,Светлана Тихановская,NaN,BBC News Русская Выборы в Беларуси в Светлана Тихановская требует издевательства над арестованным мужем,0.0000,0.0,1.000,0.000,BBC News Russian Service - Elections in Belarus: Presidential Candidate Svetlana Tikhanovskaya Demands an End to Abuse of Arrested Husband
216,216,IvanPrakop,1284348816888651776,2020-07-18 04:46:34+00:00,0,NaN,@YouTube,0,0,Сьвятлана Ціханоўская: Я іду далей | Светлана Тихановская: Я иду дальше https://youtu.be/zYTykBK1T8o через @YouTube,tikhanovskaya,NaN,NaN,@YouTube,Сьвятлана Ціханоўская Светлана Тихановская дальше https://youtu.be/zYTykBK1T8o @YouTube,0.0000,0.0,1.000,0.000,Svetlana Tikhonovskaya: I&#39;m moving on Svetlana Tikhanovskaya: I go further https://youtu.be/zYTykBK1T8o via @YouTube
339,339,VadimGuzinin,1284367768813481989,2020-07-18 06:01:53+00:00,0,NaN,NaN,0,0,BBC News Русская служба - Выборы в Беларуси: кандидат в президенты Светлана Тихановская требует прекратить издевательства над арестованным мужем,tikhanovskaya,NaN,Светлана Тихановская,NaN,BBC News Русская Выборы в Беларуси в Светлана Тихановская требует издевательства над арестованным мужем,0.0000,0.0,1.000,0.000,BBC News Russian Service - Elections in Belarus: Presidential Candidate Svetlana Tikhanovskaya Demands an End to Abuse of Arrested Husband
510,510,HUILOPUTLER,1284384822354378752,2020-07-18 07:09:39+00:00,0,NaN,NaN,0,0,"Вот мне интересно, если всё-таки Тихановская примет участие в выборах то таракану нужно фальсифицировать в огромных маштабах, пойдет он на это или зассыт?",tikhanovskaya,NaN,на это или зассыт,NaN,Вот мне интересно если всё таки Тихановская участие в выборах то таракану нужно фальсифицировать в огромных маштабах пойдет он на это или зассыт,0.0000,0.0,1.000,0.000,"So I&#39;m wondering if Tikhanovskaya does take part in the elections, then the cockroach needs to be falsified on a huge scale, will he go for it or get pissed off?"
576,576,Divnaja_Tigra,1284390116866260993,2020-07-18 07:30:41+00:00,5,NaN,NaN,0,1,"Нашей оппозиции не хватает грамотных пиарщиков. Они молодцы все, что вылезли. Тихановский сразу шум поднял.. Но теперь нужны конкретные фразы. Конкретные видео. С примерами как для дебилов, чтобы даже бабкам дошло. Потому что поверьте! Им не доходит! Им всё нормально.",tikhanovskaya,NaN,NaN,NaN,не Они молодцы все теперь нужны конкретные Конкретные С как для дебилов не доходит всё нормально,0.0000,0.0,1.000,0.000,"Our opposition lacks competent PR specialists. They are good fellows all that got out. Tikhanovsky immediately raised a fuss .. But now specific phrases are needed. Specific videos. With examples as for morons, so that even grandmothers get it. Because believe me! They don&#39;t get it! They are fine."
598,598,currrtains,1284392334231252992,2020-07-18 07:39:29+00:00,2,NaN,NaN,0,1,Светлана Тихановская икона современного феминизма,tikhanovskaya,NaN,NaN,NaN,Светлана Тихановская,0.0000,0.0,1.000,0.000,Svetlana Tikhanovskaya icon of modern feminism
633,633,euronewsru,1284396437523046400,2020-07-18 07:55:48+00:00,8,NaN,NaN,2,2,"#КУБ Белорусский ЦИК не допустил двух оппозиционных кандидатов в президенты. #Цепкало обвинил власть в трусости, а глава ЦИК говорит, что люди на улице выплескивают агрессию, т.к. не могут уехать в отпуск. https://bit.ly/2WryPST #беларусь #минск #лукашенко #бабарико #тихановская",tikhanovskaya,NaN,NaN,NaN,КУБ Белорусский ЦИК не двух кандидатов в Цепкало в трусости ЦИК говорит что люди на выплескивают т.к . не уехать в,0.0000,0.0,1.000,0.000,"#CUB The Belarusian CEC did not allow two opposition presidential candidates. # Tsepkalo accused the authorities of c

In [34]:
df_tikhanovskaya.to_csv('df_tikh.csv')

In [35]:
df_tsepkalo['translated'] = df_tsepkalo['text'].apply(lambda x: translation_googleAPI(x,'en'))

A smart man was clinging, I am sure that the fact that an equally smart woman was next to her played a role
....
#Anonymous Women taking over the #Lukashenko regime in #Belarus -Marya Kalesnikava (campaign manager of imprisoned opposition leader Viktar Babaryka) -Sviatlana Tsikhanouskaya (presidential candidate and wife of imprisoned blogger Tsikhanouski) -Veranika Tsapkala
....
LUKASHENKO LIQUIDIZED BABARIKO AND TSEPKALO. https://youtu.be/S-7LYjM5iAg via @YouTube
....
Headquarters of Tikhanovskaya, Babariko and Tsepkalo will unite in the presidential elections in Belarus
....
Headquarters of Tikhanovskaya, Babariko and Tsepkalo will unite in the presidential elections in Belarus
....
Headquarters of Tikhanovskaya, Babariko and Tsepkalo will unite in the presidential elections in Belarus
....
Headquarters of Tikhanovskaya, Babariko and Tsepkalo will unite in the presidential elections in Belarus
....
Headquarters of Tikhanovskaya, Babariko and Tsepkalo will unite in the presidential el

Veronica Tsepkalo urged people to come to the polls with homemade white bracelets. In her opinion, this will demonstrate how many people oppose the current government. And in the opinion of &quot;Belaya Rus&quot; - this is a very interesting way to turn everything in its direction.
....
https://youtu.be/-vbnfNBMXgQ # мывместе # бабарико # бабарыка #babariko # babarykа # беларусь #belarus # belaruselections2020 # беларусьвыборы2020 #humanrightsbelarus #humanrightsby #tsepkalo #tihanovskaba # славян
....
Well, everything, Tikhanovskaya, the tenacious and Kolesnikova have teamed up and I really hope that they will discuss a really important problem and decide when we can finally give menstruation to the men
....
Veronica Tsepkalo offered to come to the polls on August 9 with a homemade white bracelet on her arm. Preparing the merch with white bracelets and a flag on the other side. If everything goes cunt))
....
* A person who has something to do with Veronica Tsepkalo
....
When I grow up

You need to listen carefully, and not on emotions. Transparent hints of street protest sounded from Tsepkalo himself. Regarding &quot;criticize, offer&quot;. This is relevant for situations where the critic takes part in the decision, and in this situation we are all just a resource for their purposes.
....
Something from Tsepkalo sounded just on emotions, but there were no direct appeals. And hints - this is the case: everyone sees what he wants or does not want to see, this is not a direct appeal.
....
Yo, Lenny, Trey, June and Rio are fighting against Uncle Lenny ~ As I see it I: Tikhanovskaya, Tsepkalo, Kolesnikova and all Belarusians united and opposed the acting Yes, it&#39;s hard, he is huge, terrible and seems invincible. BUT WE ARE MORE AND WE WILL DO IT !! #FaithfulMoss
....
Beautiful #Belarus at today’s joint campaign event of Tikhanovskaya, Babariko, Tsepkalo - http://KYKY.ORG:
....
Today there was a planned, legal meeting in Minsk. At Bangalore Square. More than 7000 thous

The Supreme Court rejected the complaint of the politician, entrepreneur, former head of the Hi-Tech Park Valeriy Tsepkalo
....
Veronica Tsepkalo: &quot;The government spends our taxes on the purchase of residences, planes, cars&quot;
....
You do not need, but it will be very useful for thousands of doubters, especially newcomers, to make sure of this. And if the platform, which Tsepkalo seems to be talking about, is also ready, then this fact will also be documented in a sense. This is what we are talking about.
....
The Supreme Court did not satisfy Tsepkalo&#39;s complaint about the refusal to register. https://news.tut.by/economics/693471.html?tw
....
138. The Supreme Court dismissed the complaint of Valery Tsepkalo about the refusal to register as a candidate for the presidency of the Republic of Belarus
....
Supreme Court makes final decision on registration of Tsepkalo as presidential candidate
....
The court rejected Tsepkalo&#39;s complaint about non-registration as a presiden

The headquarters of Babariko, Tsepkalo and Tikhanovskaya are trying to lower the degree of protest of the Belarusian society, urging it to participate in pseudo-elections. Instead of organizing an alternative vote, they want to create a system for recording fraud. But somehow the dictator himself admitted to falsifications.
....
This is a myth that Luke came up with. And by the way Babaryk or Tsapkala repeated it, if I&#39;m not mistaken. That there was a radical opposition, waved bbb, wanted to expel all Muscovites and make everyone speak their bird language instead of the normal Russian language.
....
At one o&#39;clock in the morning I burst into tears while watching the video from the action by Tikhanovskaya, Tsepkalo, Kolesnikova Incredible things are happening !!!
....
You did not answer all the questions of the presenter, or not all of them got on the air. I especially remember the reasons for coming out of the shadows under the shadow of the leader babariko / clinging right now

...

And to live, you need just money. And the professional opposition itself is begging and has no plans on how to earn this money for us. Babariko and Tsepkalo in this respect stood out favorably for the better, the people followed them.
....
&quot;We stripped Lukashenka, made him an outcast.&quot; Tsepkalo gave his first interview after ... https://belaruspartisan.by/politic/507023/?utm_source=twitter&amp;utm_medium=cpc
....
I hope you won&#39;t vote for the tenacious more?
....
Lukashenko opponent Valery Tsepkalo flees to Russia fearing arrest in Belarus https://on.rt.com/amkj
....
Why do we need this cannibal opinion? Well, if Putin is an authority for Tsepkalo, then let him remain in Russia, we are not on our way, we see ourselves with the free world.
....
is it true that Tsepkalo left for Russia?
....
“Don&#39;t help”: Belarusian opposition leader Valery Tsepkalo appealed to the Russian authorities after leaving for Moscow:
....
Who did you hear about grants from? For bt from the evi

Unregistered Opposition Presidential Candidate Tsapkala, Children Flee Belarus 
....
Valey Tsepkalo: &quot;I can imagine how Putin will look at him after he draws voices for himself&quot;
....
I don&#39;t know who Tsepkalo is, but Tikhanovsky is definitely an asshole
....
People are not fighting for Tsepkalo, but for the right to choose. Is it really so difficult to understand the elementary? Let Poznyak move out in six months, if he doesn&#39;t get cold feet and return. True, I&#39;m afraid the result will not please him ...
....
OK. The cling is not what you need. Give another surname that suits you.
....
Fakes again. This time about the ongoing work to deprive Tsepkalo of parental rights https://newsgomel.by/news/society/snova-feyki-na-sey-raz-o-provodimoy-rabote-po-lisheniyu-roditelskikh-prav-tsepkalo_52066.html
....
&quot;We have stripped Lukashenka, made him an outcast.&quot; Oppositionist Tsepkalo gave his first interview after his escape https://bit.ly/2WVoHlp
....
&quot;You wi

And how was he allowed there if they wrote that he had a covid? Or the same pizdabol clinging to the Lukashists?
....
Tsepkalo remembered about Masquerade. The startup and the product appeared at the hackathon in Imaguru. How did Tsepkalo create Masquerade? who is in the subject?
....
I wrote about Tsepkalo&#39;s interview with Meduza half an hour before it was posted. I take orders to predict the future.
....
5) If we talk about the agenda, gentlemen, superficial sofa chroniclers reproach at the moment the Tsepkalo family and the Tikhonovsky family for the fact that these people have done everything they can at the moment in order to protect their families Yes, Russia is far from the best option, but somehow
....
Do you remember how I said that they are one team? Officially, the union was after the announcement of the results, but here it turns out that the decision was made in advance. Excerpt from Tsepkalo&#39;s interview with Meduza
....
It was for this purpose that Tsepkalo introd

There you have a little bit of connection with reality, the boy ran away to Rashka and asks the Russian authorities not to help the Belarusian authorities with words ... #Belarus #tvrain
....
Elections in Belarus: Luka has already lost https://youtu.be/WVFjGVwaeTI via @YouTube - NEW PANDEMIA COVERED BELARUS: https://youtu.be/28Aj6z1EPRQ - Tsepkalo told about the reasons for his escape to Russia ....: https: / /youtu.be/rov_c0WFFeI - Truth, about FALSIFICATIONS &quot;Father&quot;: https://youtu.be/DxHuGJWEKow
....
Tsepkalo, after fleeing to Moscow, publicly addressed the Russian authorities
....
Ex-presidential candidate Tsepkalo said that now it is dangerous for him to be in Belarus
....
Tsepkalo said that now it is dangerous for him to be in Belarus Former presidential contender, former head ...
....
Belarusian opposition leader Tsepkalo said that now it is dangerous for him to be in Belarus • ANTIKOR portal
....
Answer yourself: can you become the same? This light is inside, and it c

Tsepkalo invited Domracheva to the Committee of National Unity, but she likes a different committee
....
Doesn&#39;t it seem strange that Tsepkalo is hiding in Moscow?
....
I also ordered T-shirts from Minsk I am crying ... p. I draw symbols of agitation on your T-shirts for 10 rubles # Tihanovskaya # Babariko # cling
....
1. Fantasy. Both about my words and about Tsepkalo. 2. Why now we can not join forces on those issues that are needed for both? The dictator develops mova, sambali, what is important to you? Not! The dictator destroyed the economy, which is important to me. Why don&#39;t we get the dictator down first?
....
It&#39;s so strange, Tsepkalo here gives the game interview on RBC. In Russia, he now, ran away from Luka so that he would not plant, that he would not hand over chtoli? And with this he somehow whitewashes the local ayrod. The kind accepted. What the heck? Do not know?
....
&quot;Do not help&quot;: Belarusian oppositionist Valery Tsepkalo appealed to the Russian 

In [ ]:
df_tsepkalo.head(20)

In [36]:
df_tsepkalo.to_csv('df_tsep.csv')

In [37]:
df_babariko['translated'] = df_babariko['text'].apply(lambda x: translation_googleAPI(x,'en'))

Shapito circus. After Babariko was closed, elections ended
....
What is the point of holding a referendum at the same time as the presidential campaign? It is obvious that Babaryka and his staff have thus created a &quot;path of retreat&quot;, as they still expect to stay in politics regardless of the results of the election campaign.
....
The court rejects the complaint about the refusal to register Babariko as a presidential candidate
....
Yesterday the headquarters of the presidential candidate Svetlana Tikhanovskaya joined forces with the teams of Valery Tsepkalo and Viktor Babariko, who were not registered to participate in the elections. At the press conference, all three headquarters were represented by women. Now they are compared to superheroines
....
For me, it&#39;s time of year - Sasha Zvereva and Edik Babaryka
....
Babariko&#39;s lawyers appealed to the Supreme Court against the refusal to register as a candidate https://afn.by/news/i/277893
....
Three women against Lukash

They can only spoil everything. Well, zmagars in general. I was glad when Babariko walked. And then the zmagars wanted to get into this movement
....
“𝗜 𝗹𝗼𝘃𝗲 𝗲𝘃𝗲𝗿𝘆 𝘀𝗶𝗻𝗴𝗹𝗲 𝗰𝗼𝗿𝗻𝗲𝗿 𝗼𝗳 𝗕𝗲𝗹𝗮𝗿𝘂𝘀" According to numerous polls from independent organizations, Viktar Babaryka holds over 50% of the vote. #Belarus #freebelarus https://www.instagram.com/p/CC0ZMSeFLpl/?igshid=1ld01wdbt6stm
....
They are melting for Babariko
....
In babariko. U Tikhonovskaya v inste
....
#Babaryko was denied his constitutional rights to be represented by a lawyer in appeal against the decision not to register him as a candidate. The complaint, signed by the lawyer on behalf of his client, was not accepted by the Supreme Court. @kykyorg
....
For Russians in Tallinn, Babariko is a political prisoner. For 18 countries of Europe and Belarus, he is a corrupt official. And for the Estonian special services - new faces in the base of unreliable
....
United female team of #Tsihanouskaya, #Tsapkala and #Kalesnikava (team of #B

Mishustin handed over a black mark from Pyni Luka ... for Babariko&#39;s bank ... it was not we who said it, but the bankers ordered you ... at the next opening of the bridge across the Sozh or visiting the &quot;Put Ilyich&quot; collective farm, you will crash like a partridge
....
There must be a strategy and a course of action to be followed. At least general. But it is not that Tikhanovsky is first for ignoring the elections, then in the end Tikhanovskaya says that we are going to the polls. Babariko half said the constitution in his extreme video. And what next?
....
It seems to me that my avatar looks like Babariko
....
Lukashenka said that Babariko and about 30 other people began to testify: “Everyone is testifying. And this one (Babariko. - note from Gazeta.Ru) has already burst into tears. &quot; https://www.gazeta.ru/politics/news/2020/07/19/n_14691583.shtml
....
Lukashenka announced Babariko who started to testify
....
Where is the photo of the sobbing Babariko?
....
Lukashe

See the last part of my text - I am afraid, it will not be smooth. Still, it is currently Tsichanovska, not Babaryka, who leads the protest.
....
Lukashenko said that his arrested rival Babariko &quot;sobbed&quot; and began to testify in the Belgazprombank case
....
Lukashenka said that Babariko began to testify ”to RF-media. Only the latest news!:
....
Lukashenka said that his arrested rival Babariko &quot;sobbed&quot; and began to testify
....
Is Babariko a Kremlin candidate?
....
EchoMSK: Lukashenka on the Babariko and Belgazprombank case: &quot;Everyone is testifying, and this one has already burst into tears&quot;
....
&quot;Lukashenka said that his arrested rival Babariko&quot; burst into tears &quot;and began to testify in the Belgazprombank case&quot; - I can imagine how the man was tortured there.
....
“Small miracle” - A letter from Babariko&#39;s son came to another address. But the girl got it anyway https://telegraf.by/politika/malenkoe-chudo-pismo-ot-syna-babariko-prishlo

Lukashenko: Babariko began to cry and testify http://dlvr.it/Rc0H6Z
....
I called my retired mom in Mogilev, I say, here they say Gaidukevich said ... And who else is she? Better tell me, when is the next picket behind Tikhanovskaya and what Babariko has?
....
&quot;War does not have a woman&#39;s face.&quot; Of the three most popular rivals of Lukashenko in the upcoming presidential elections, the CEC registered only Svetlana Tikhanovskaya. The election headquarters of candidates Valery Tsepkalo and Viktor Babariko decided to support her.
....
&quot;President of the Republic of Belarus: We demand the release of Viktor Babariko&quot; - Sign the petition! http://chng.it/sf8vjjP5 via @changeorgrus
....
Three women against Lukashenka. Interview with Svetlana Tikhanovskaya, wife of Valery Tsepkalo Veronika and representative of Victor Babariko&#39;s headquarters Maria Kolesnikova
....
&quot;President of the Republic of Belarus: We demand the release of Viktor Babariko&quot; - Sign the peti

...

Part of EU taxpayers 6 millions will go directly to #Lukashenka regime police ostensibly “to fight organized crime” I think, after EU training, the investigators working on the case of prisoner of conscience Viktar Babaryka will learn to interrogate him more effectively 
....
Babariko really said that any association, IF IT IS FOR THE GOOD OF PEOPLE, he is for. But is the union with rfia good for the people? On the second paragraph. There is no need to lie, he did not say that &quot;we should rejoice.&quot; You are the same as the propogandon bows.
....
Babariko is not a pro-Russian candidate, he is a good man, and you can clearly see this because Luka put him in prison
....
Babariko, of course.
....
Babariko&#39;s headquarters applied for the texts of the propagandist of the presidential newspaper Mukovozchik
....
Babariko spoke about the reality of Ceausescu&#39;s scenario. During those events, 1000 people died. Does this mean that Babariko is intimidating people?
....
Babariko&#39;s

The Prosecutor General&#39;s Office announced a new charge to Babariko - illegally received and divided 28 million rubles with his accomplices -
....
Babariko was charged with new bribe-taking on an especially large scale: http://go.tass.ru/wDHW
....
The Prosecutor General&#39;s Office for the first time told what exactly Viktor Babariko is accused of
....
Babariko&#39;s headquarters filed an application for Mukovozchik&#39;s articles in the UK and the prosecutor&#39;s office https://4esnok.by/novosti/shtab-babariko-podal-zayavlenie-na-stati-mukovozchika-v-sk-i-prokuraturu/ Viktor Babariko’s headquarters expressed indignation at Andrei’s articles A flour carrier, which is regularly published by “SB-Belarus Today”. In this regard, supporters.
....
The accusation against Viktor Babariko was reclassified from “giving a bribe” to “accepting values by an official as part of an organized group”. They were looking for a secret room in the house, and in the end they found valuables in deposit 

#Belarus'ian authorities reindict imprisoned presidential hopeful #Babaryka https://belsat.eu/en/news/belarusian-authorities-reindict-imprisoned-presidential-hopeful/?btid=d5b539c6770e46a30bce2cff66194ba8 #Election2020
....
What unexpected news, the KGB man fled to Moscow. He didn’t want to become a real &quot;oppositionist&quot;, having shared the camera with his office comrade Babariko.
....
Viktor Babariko faces new charges → https://grodno.in/news/34621
....
There is no Babariko headquarters. There is no presidential candidate with the name Babariko.
....
Babariko&#39;s headquarters said to the prosecutor&#39;s office against the author of the presidential newspaper, who wrote about the deaths and marks of Satan - http://KYKY.ORG:
....
According to the Prosecutor&#39;s Office, Wiktar Babaryka did not accept, but gave bribes
....
Babariko&#39;s lawyer says that a new charge has been brought against Babariko: “The press service of the prosecutor’s office, apparently, can comment on s

Norway introduces quarantine for arrivals from Spain. The Prosecutor General&#39;s Office of Belarus has reclassified the charges against Babariko. Italian MPs approved March 18 as a day of remembrance for the victims of the coronavirus. https://ik.arhano.ru/archives/58249 Norway introduces quarantine for arrivals from Spain No ...
....
In #Orsha, #Viciebsk region, #Belarus, at least 1,000 people attended the campaign rally by opposition presidential candidate Sviatlana Tsikhanouskaya. Viktor Tsoy’s late 1980’s song Changes! can be heard at the background: 
....
Babariko&#39;s lawyer commented on the CEC&#39;s response to the complaints of Belarusians about the limitation of the number of observers - Brestskaya Gazeta https://www.bg.by/news/advokat-babariko-prokommentiroval-otvet-cik-na-zhalobyi-belorusov-po-povodu-ogranicheniya- chisla-nablyudateley /
....
Lukashenko regularly recalled him and Babariko on TV, but who they were I heard very distantly and I confess I had to google. But 

Belarus expects from the shareholders of &quot;Belgazprombank&quot; - Russian &quot;Gazprom&quot; and &quot;Gazprombank&quot; - proposals on the new composition of the board of the Belarusian bank. The former head of the bank, Babariko, decided to participate in the presidential elections, but was arrested. July 25, 2020.http: //Interfax.ru
....
I have a complete feeling that it would be better for Lukashenko to leave Tsepkalo or Babariko
....
Belarus expects from the shareholders of &quot;Belgazprombank&quot; - Russian &quot;Gazprom&quot; and &quot;Gazprombank&quot;, proposals for a new composition of the board of the Belarusian bank. Former bank chief Babariko decided to run in the presidential election but was arrested. July 25, 2020. http://Interfax.ru
....
now it&#39;s already strange to remember 2010, when, having dispersed 50k people in an hour, they scared the whole people so much that then for 5 years people were afraid to leave. Then we felt like an opposition, a minority. No

In [38]:
df_babariko.head(20)

,Unnamed: 0,username,tweet_id,date,favorites,hashtags,mentions,retweets,replies,text,topical_persons,topical_orgs,persons,orgs,propn,compound,neg,neu,pos,translated
186,186,Anton85376290,1284340286710255616,2020-07-18 04:12:40+00:00,0,NaN,NaN,0,0,"Цирк шапито. После того как закрыли Бабарико, выборы закончились",babariko,NaN,NaN,NaN,Цирк шапито как закрыли Бабарико выборы закончились,0.0,0.0,1.0,0.0,"Shapito circus. After Babariko was closed, elections ended"
243,243,bioby6,1284353651407020032,2020-07-18 05:05:47+00:00,0,NaN,NaN,0,1,"Які ўвогуле сэнс адначасова з прэзідэнцкай кампаніяй займацца нейкім рэферэндумам? Відавочна, што Бабарыка і ягоны штаб такім чынам стварылі ""шлях адступлення"", бо яны ўсё ж такі разлічываюць застацца ў палітыцы па-за залежнасцю ад вынікаў выбарчай кампаніі.",babariko,NaN,NaN,выбарчай кампаніі,займацца нейкім рэферэндумам Відавочна што Бабарыка і ягоны штаб такім чынам стварылі шлях адступлення бо яны ўсё застацца ў палітыцы па за залежнасцю ад вынікаў выбарчай кампаніі,0.0,0.0,1.0,0.0,"What is the point of holding a referendum at the same time as the presidential campaign? It is obvious that Babaryka and his staff have thus created a &quot;path of retreat&quot;, as they still expect to stay in politics regardless of the results of the election campaign."
254,254,culptuben90,1284355250875498497,2020-07-18 05:12:08+00:00,0,NaN,NaN,0,0,Суд отклонил жалобу на отказ в регистрации Бабарико кандидатом в президенты,babariko,NaN,Бабарико,NaN,Суд отклонил жалобу на в Бабарико кандидатом в,0.0,0.0,1.0,0.0,The court rejects the complaint about the refusal to register Babariko as a presidential candidate
280,280,directklad,1284358068277579776,2020-07-18 05:23:20+00:00,20,NaN,NaN,3,0,"Вчера штаб кандидата в президенты Белоруссии Светланы Тихановской объединил усилия с командами Валерия Цепкало и Виктора Бабарико, которых не зарегистрировали для участия в выборах. На пресс-конференции все три штаба представляли женщины. Теперь их сравнивают с супергероинями",babariko,NaN,NaN,NaN,кандидата в Белоруссии Светланы Тихановской усилия Валерия Цепкало Виктора Бабарико которых для в На конференции все три штаба представляли Теперь сравнивают с супергероинями,0.0,0.0,1.0,0.0,"Yesterday the headquarters of the presidential candidate Svetlana Tikhanovskaya joined forces with the teams of Valery Tsepkalo and Viktor Babariko, who were not registered to participate in the elections. At the press conference, all three headquarters were represented by women. Now they are compared to superheroines"
448,448,paddubitskaya,1284376691314393088,2020-07-18 06:37:20+00:00,4,NaN,NaN,0,0,Для мяне пара года — Саша Зверава ды Эдзік Бабарыка,babariko,NaN,Саша Зверава,NaN,Для Зверава ды Эдзік,0.0,0.0,1.0,0.0,"For me, it&#39;s time of year - Sasha Zvereva and Edik Babaryka"
539,539,_afnby,1284387286872993792,2020-07-18 07:19:26+00:00,0,NaN,NaN,0,0,Адвокаты Бабарико обжаловали в ВС отказ в регистрации кандидатом https://afn.by/news/i/277893,babariko,NaN,NaN,ВС,Адвокаты Бабарико в ВС отказ в кандидатом,0.0,0.0,1.0,0.0,Babariko&#39;s lawyers appealed to the Supreme Court against the refusal to register as a candidate https://afn.by/news/i/277893
577,577,opd_voshod,1284390192586076160,2020-07-18 07:30:59+00:00,0,NaN,NaN,0,0,Три женщины против Лукашенко. Как штабы Цепкало и Бабарико поддержали Тихановскую - https://voshod.org/videochanel/newschanels/%d1%82%d1%80%d0%b8-%d0%b6%d0%b5%d0%bd%d1%89%d0%b8%d0%bd%d1%8b-%d0%bf%d1%80%d0%be%d1%82%d0%b8%d0%b2-%d0%bb%d1%83%d0%ba%d0%b0%d1%88%d0%b5%d0%bd%d0%ba%d0%be-%d0%ba%d0%b0%d0%ba-%d1%88%d1%82%d0%b0%d0%b1/,babariko,NaN,NaN,NaN,Три Лукашенко Как штабы Цепкало Бабарико Тихановскую,0.0,0.0,1.0,0.0,Three women against Lukashenka. How the headquarters of Tsepkalo and Babariko supported Tikhanovskaya - https://voshod.org/videochanel/newschanels/%d1%82%d1%80%d0%b8-%d0%b6%d0%b5%d0%bd%d1%89% d0% b8% d0% bd% d1% 8b-% d0% bf% d1% 80% d0% be% d1% 82% d0% b8% d0% b2-% d0% bb% d1% 83% d0% ba% d0% b0% d1% 88% d0% b5% d0%

In [39]:
df_babariko.to_csv('df_bab.csv')

In [40]:
df_lukashenko['translated'] = df_lukashenko['text'].apply(lambda x: translation_googleAPI(x,'en'))

Just disqualify Lukashenko. 
....
CORONOVIRUS is not only the need to remove unnecessary people, but also the LOSS OF BELARUS, USA FOR TERRORISM IN LIFE TURCHENKO T.O. DOES NOT GIVE IN A DEBT TO BELARUS, SO THE KGB OF BELARUS WITH V.V. PUTIN WANT TO TAKE LUKASHENKO A.G., HIS BAND of 6 PERSONS on a ZHIVTSA, on the Death of A.G. TURCHENKO, an innocent
....
ZECOMAND&#39;S FAILURE! WAR WITH A BALL. LUKASHENKO WILL NOT ALLOW ZE INTERFERENCE ... https://youtu.be/0pXpH0IsMIQ via @YouTube
....
THE MONEY OF BELARUS, USA FOR TERRORISM IN LIFE TURCHENKO T.O. DOES NOT GIVE IN A DEBT TO BELARUS, SO THE KGB OF BELARUS WITH V.V. PUTIN WANT TO TAKE LUKASHENKO A.G. and HIS BAND of 6 PERSONS on the ZHIVTSA, on the Death of A.G. TURCHENKO, an innocent person, she was played as a Victim of the NKVD-MVD all my life from Childhood
....
THE MONEY OF BELARUS, USA FOR TERRORISM IN LIFE TURCHENKO T.O. DOES NOT GIVE IN A DEBT TO BELARUS, SO THE KGB OF BELARUS WITH V.V. PUTIN WANT TO TAKE LUKASHENKO A.G. and HIS 

Dying convulsions ... I wonder who will last longer, the wisest, or Lukashenka ?!
....
Prime Minister of the Republic of Kazakhstan A. Mamin took part in the meeting of the UMPS in Minsk and held a meeting with the President of the Republic of Belarus A. Lukashenko
....
Lukashenko promised to tell Mishustin about “strange things” in the elections | | http://LIXL.RU
....
it was he who cheated Lukashenka, a plagiarist of dicks
....
Vadim Prokopyev&#39;s video message to Lukashenka gets over 600 thousand views per day
....
Lukashenko told Mishustin about the short-lived unequal union
....
Lukashenko apologized to Mamin for waiting https://tengrinews.kz/news/lukashenko-izvinilsya-pered-maminyim-za-ojidanie-408684/
....
Lukashenka apologized to Mamin for waiting
....
The people and Lukashenka must be reversed - he is in prison, and the people to freedom. Such a simple scheme ...
....
Have you sniffed a lot of gunpowder yourself? He served as a political guy as a partner: &quot;From 1975 to 

Lukashenka apologized to Mamin for waiting http://dlvr.it/RbsLz1
....
Lukashenko told Mishustin that strange things happen during the presidential campaign https://virtualbrest.by/news80922.php Russian Prime Minister Mikhail Mishustin believes that Russia and Belarus could take a number of new measures to unleash the potential of the Treaty on the Union of States #brest_by
....
The dictatorship fell. Terpily p ... fuck cops. Minsk, Eburg, Khabarovsk https://kungurov.livejournal.com/272655.html?utm_source=twsharing&amp;utm_medium=social # protest # Putinism # Tikhanov&#39;s # opposition # dictatorship # Lukashenko # revolution # Belarus # Khabarovsk # Eburg
....
Lukashenko promised to tell Mishustin about “strange things” taking place in Belarus “Today it is impossible, as in the Middle Ages, to seize a country and subjugate for 100-150 years”. https://rusvesna.su/news/1595050265
....
Prime Minister Kubatbek Boronov met with President of Belarus Alexander Lukashenko
....
Lukashenko: Rus

The red stick from Mishustin&#39;s nose grows right into Lukashenka&#39;s mouth. Yes, it&#39;s strange.
....
Lukashenko promised to tell Mishustin about “strange things” taking place in Belarus (+ VIDEO, PHOTO) “Today it is impossible, as in the Middle Ages, to seize a country and subjugate it for 100-150 years”. https://rusvesna.su/news/1595050265
....
Two days before Mishustin&#39;s visit to Minsk on Belarus 1, a story was released about the involvement of Russian Deputy Finance Minister Andrei Kruglov in money laundering at Belgazprombank. Let&#39;s tell why Lukashenka refers to the topic of Russia&#39;s interference in the elections. https://gazeta.ru/politics/2020/07/17_a_13155919.shtml
....
Well, Hodor talks about this in an interview with Dudu. But I’ll do it even easier. Will Lukashenka win honestly with 3%? So why did Yeltsin manage with 4%?
....
Lukashenko promised to tell Mishustin about “strange things” taking place in Belarus (+ VIDEO, PHOTOS) | Russian spring
....
And Pre

Lukashenko showed his fear: why practically all of Belarus hates the president By removing the rating ...
....
Khabarovsk, Belarus, Putin, Lukashenko, Ukraine, USA, war, Khazan, Denisenko, 2 days view? usp = sharing https://vk.com/video?section=upload&amp;z=video16204738_456241525%2Ffa0976cedafe4c1e53
....
Lukashenko hinted at a break in &quot;friendship&quot; with Russia and set a condition for Putin • ANTIKOR portal
....
Yes it is all over the world. We almost have a &quot;Ukrainian trace&quot;, Lukashenka has a &quot;Russian trace&quot; :)))) Some pathfinders are all around :)))
....
Who is for Lukashenka is Philip Kirkorov.
....
well, also Lukashenko P ..... fits, of course
....
Lukashenko to Pashinyan: “You were the engine of this meeting there” - video
....
One question - where and when did Lukashenka&#39;s children serve - dmitry, viktor, nikolai?
....
Russia has its own dictator, why do they constantly rub in on Lukashenka !?
....
This year I have not met a single person who wo

...

Women unite in maverick attempt to unseat Lukashenko in Belarus 
....
Mom: show me what drawings you want to take to the psychologist tomorrow. I: * proudly present several works in watercolors and Kolya Lukashenko * Ma: not well, that&#39;s clear, but who is there from the famous ??? I: YES, IN THE MEANING OF I HAVE YOU HOW MANY TIMES TOLD YOU THAT I&#39;M LEAVING TO KOLE LUKASHENKO WHAT IS NOT FAMOUS?
....
Khabarovsk is revolting! Lukashenka&#39;s great love. The 1.8 Trillion Euro Party ... https://youtu.be/Klwnszh4zc0 via @YouTube
....
#Lukashenko has jailed two of his main election rivals and detained hundreds of protesters in a crackdown on dissent against his 26-year rule in #Belarus 
....
It looks like the Belarusians have decided. Taki to zero Lukashenka.
....
Khabarovsk is revolting! Lukashenka&#39;s great love. The 1.8 Trillion Euro Party ... https://youtu.be/Klwnszh4zc0 via @YouTube
....
The new President of the Republic of Belarus! If Lukashenko does not kill her.
....
Yes,

Watch &quot;Khabarovsk is revolting! Lukashenko&#39;s great love. The party for 1.8 trillion euros -&quot; Zapovednik, issue 133 &quot;on YouTube
....
Americans elect everyone in Belarus, including Lukashenko himself, and he is hired not by the Belarusian people, but by the United States, and the American authorities need to convince him to hire him as president. Evgeny Fedorov
....
lukashenko sieby from belarus in brotherly #MVTHottest BTS @BTS_twt
....
The power of Lukashenka is given by the occupying American regime. Evgeny Fedorov
....
The gang of Lukashenko A.G. is an abnormal special forces. no one openly does Murder and Violence under the open Sanctions for terrorism and human trafficking, everyone watches as after operations and doctors the rapists turn on the device and Continue destruction with vibrations, knowing that Death for Terrorism awaits them
....
Urgently..! The RATING of Tikhanovskaya is named ..! Lukashenka will be SHARED ..! Last ... https://youtu.be/zp766mAHMjw v

Lukashenka, running for a sixth term, unexpectedly faced a powerful adversary http://dni.ru/polit/2020/7/26/455534.html
....
Three short videos that will give you a feel for the atmosphere of the rally of Lukashenka&#39;s opponents in Gomel https://nn.by/?c=ar&amp;i=255976&amp;lang=ru
....
Resignation of the President of the Republic of Belarus - A.G. Lukashenko - Sign the Petition! http://chng.it/Y6KmdJyb via @Change
....
And here, fucking, Lukashenko is not the president, it is interesting, of course
....
Great job. Women unite in maverick attempt to unseat Lukashenko in Belarus 
....
Khabarovsk is revolting! Lukashenka&#39;s great love. The 1.8 Trillion Euro Party ... https://youtu.be/Klwnszh4zc0 via @YouTube
....
So only Lukashenka is shown on TV
....
Tikhanovskaya held a meeting of thousands of Lukashenka&#39;s opponents in Homel Video, photo:
....
Do not chat up the dispute with your Lukashenka.
....
Has Lukashenko already found a house in Rostov?
....
My understanding is that Lu

&quot;Look&quot; Khabarovsk is revolting! Lukashenka&#39;s great love. € 1.8 Trillion Lot - Preserve, issue 133 on YouTube
....
Can anyone imagine such an atmosphere at a meeting with Lukashenka ?! Without coercion, without intimidation with the deprivation of bonuses and threats of dismissal?! So that people just take it and come. And like this? Personally, I can&#39;t imagine it! Better than 1000 different ratings!
....
My friend Sasha in Homiel sent me this photo of the rally tonight in support of Tsikhanovskaya. I never thought I’d see scenes like this in Lukashenka-land. #Belarus 
....
“To the President of the Republic of Belarus AG Lukashenko: Stop extermination of dogs by this method! Help reduce their numbers without killing! &quot; - Sign the petition! http://chng.it/yWvwSQJB via @changeorgrus
....
Belarusian President Lukashenko refused an attractive offer from the International Monetary Fund because the conditions included the imposition of the lockdown.
....
Taisiya Povaliy

Kutsenko is a real man. And Kirkorov kissing Lukashenko&#39;s hand is a schmuck.
....
Relax, Belarusian people, for Lukashenko, you will elect Lukashenko again by an overwhelming majority, but the younger one. Democracy is all based on elections
....
Lukashenko Vladimir Vladimirovich
....
Three women alliance in Belarus attempts to end Lukashenko's presidency of 26 years 
....
How oh @ eno in Belarus! I hold my fists for them so that everything works out for them! Lukashenko has already lost anyway)
....
If you arrange the distribution of food packages, half of Minsk will crush each other at pickets for Lukashenka. Trucks with buckwheat, beautiful girls, like grid gels in the old formula-1, a selfie zone, Alivaria at a discount. It&#39;s just that it&#39;s not me in his team, but soviet dinosaurs, so there will be 10 people.
....
Women unite in maverick attempt to unseat Lukashenko in Belarus 
....
Belarusians, take care of Lukashenka Today the issue of the country&#39;s sovereignty is

In [41]:
df_lukashenko.head()

,Unnamed: 0,username,tweet_id,date,favorites,hashtags,mentions,retweets,replies,text,topical_persons,topical_orgs,persons,orgs,propn,compound,neg,neu,pos,translated
3,3,32iAu,1284277375954374656,2020-07-18 00:02:41+00:00,0,NaN,NaN,0,0,Just disqualify Lukashenko.,lukashenko,NaN,Lukashenko,NaN,Lukashenko,0.0,0.0,1.0,0.0,Just disqualify Lukashenko.
6,6,Tamara50219545,1284277497778130945,2020-07-18 00:03:10+00:00,1,NaN,NaN,0,0,"КОРОНОВИРУС-это не только нужно убрать ненужных людей, но еще БЕЗДЕНЕЖЬЕ БЕЛАРУСИ, USA ЗА ТЕРРОРИЗМ НА ЖИЗНИ ТУРЧЕНКО Т.О. НЕ ДАЕТ В ДОЛГ БЕЛАРУСИ, ПОЭТОМУ КГБ БЕЛАРУСИ С ПУТИНЫМ В.В. ХОТЯТ ВЗЯТЬ ЛУКАШЕНКО А.Г.,ЕГО БАНДУ из 6 ЧЕЛОВЕК на ЖИВЦА, на Смерти ТУРЧЕНКО А.Г., невиновного",lukashenko,NaN,NaN,Смерти ТУРЧЕНКО А.Г.,КОРОНОВИРУС это не только нужно убрать ненужных людей но еще БЕЛАРУСИ USA ЗА ТЕРРОРИЗМ НА ЖИЗНИ ДАЕТ В ДОЛГ БЕЛАРУСИ ПОЭТОМУ КГБ С ПУТИНЫМ В.В. БАНДУ из ЧЕЛОВЕК ЖИВЦА на Смерти ТУРЧЕНКО А.Г.,0.0,0.0,1.0,0.0,"CORONOVIRUS is not only the need to remove unnecessary people, but also the LOSS OF BELARUS, USA FOR TERRORISM IN LIFE TURCHENKO T.O. DOES NOT GIVE IN A DEBT TO BELARUS, SO THE KGB OF BELARUS WITH V.V. PUTIN WANT TO TAKE LUKASHENKO A.G., HIS BAND of 6 PERSONS on a ZHIVTSA, on the Death of A.G. TURCHENKO, an innocent"
7,7,SKRIPOLEGWOLF,1284277562764664832,2020-07-18 00:03:26+00:00,0,NaN,@YouTube,0,0,ПРОВАЛ ЗеКОМАНДЫ! ВОЙНА С ШАРИЕМ. ЛУКАШЕНКО НЕ ДОПУСТИТ ВМЕШАТЕЛЬСТВА ЗЕ... https://youtu.be/0pXpH0IsMIQ через @YouTube,lukashenko,NaN,NaN,@YouTube,ПРОВАЛ ЗеКОМАНДЫ ВОЙНА С ШАРИЕМ ЛУКАШЕНКО НЕ ДОПУСТИТ ВМЕШАТЕЛЬСТВА ЗЕ https://youtu.be/0pXpH0IsMIQ @YouTube,0.0,0.0,1.0,0.0,ZECOMAND&#39;S FAILURE! WAR WITH A BALL. LUKASHENKO WILL NOT ALLOW ZE INTERFERENCE ... https://youtu.be/0pXpH0IsMIQ via @YouTube
8,8,Tamara50219545,1284277856537894912,2020-07-18 00:04:36+00:00,0,NaN,NaN,0,0,"БЕЗДЕНЕЖЬЕ БЕЛАРУСИ, USA ЗА ТЕРРОРИЗМ НА ЖИЗНИ ТУРЧЕНКО Т.О. НЕ ДАЕТ В ДОЛГ БЕЛАРУСИ, ПОЭТОМУ КГБ БЕЛАРУСИ С ПУТИНЫМ В.В. ХОТЯТ ВЗЯТЬ ЛУКАШЕНКО А.Г. и ЕГО БАНДУ из 6 ЧЕЛОВЕК на ЖИВЦА, на Смерти ТУРЧЕНКО А.Г., невиновного человека,ней играли как Жертве НКВД-МВД всю жизнь с Детства",lukashenko,NaN,NaN,Смерти ТУРЧЕНКО А.Г.,БЕЗДЕНЕЖЬЕ БЕЛАРУСИ USA ЗА ТЕРРОРИЗМ НА ЖИЗНИ ДАЕТ В ДОЛГ БЕЛАРУСИ ПОЭТОМУ КГБ С ПУТИНЫМ В.В. ЛУКАШЕНКО А.Г. ЕГО БАНДУ ЖИВЦА на Смерти ТУРЧЕНКО А.Г. ней как Жертве НКВД МВД с Детства,0.0,0.0,1.0,0.0,"THE MONEY OF BELARUS, USA FOR TERRORISM IN LIFE TURCHENKO T.O. DOES NOT GIVE IN A DEBT TO BELARUS, SO THE KGB OF BELARUS WITH V.V. PUTIN WANT TO TAKE LUKASHENKO A.G. and HIS BAND of 6 PERSONS on the ZHIVTSA, on the Death of A.G. TURCHENKO, an innocent person, she was played as a Victim of the NKVD-MVD all my life from Childhood"
9,9,Tamara50219545,1284277934350577666,2020-07-18 00:04:54+00:00,0,NaN,NaN,0,0,"БЕЗДЕНЕЖЬЕ БЕЛАРУСИ, USA ЗА ТЕРРОРИЗМ НА ЖИЗНИ ТУРЧЕНКО Т.О. НЕ ДАЕТ В ДОЛГ БЕЛАРУСИ,ПОЭТОМУ КГБ БЕЛАРУСИ С ПУТИНЫМ В.В. ХОТЯТ ВЗЯТЬ ЛУКАШЕНКО А.Г. и ЕГО БАНДУ из 6 ЧЕЛОВЕК на ЖИВЦА, на Смерти ТУРЧЕНКО А.Г., невиновного человека,ней играли как Жертве НКВД-МВД всю жизнь с Детства",lukashenko,NaN,NaN,Смерти ТУРЧЕНКО А.Г.,БЕЗДЕНЕЖЬЕ БЕЛАРУСИ USA ЗА ТЕРРОРИЗМ НА ЖИЗНИ ДАЕТ В ДОЛГ БЕЛАРУСИ ПОЭТОМУ КГБ С ПУТИНЫМ В.В. ЛУКАШЕНКО А.Г. ЕГО БАНДУ ЖИВЦА на Смерти ТУРЧЕНКО А.Г. ней как Жертве НКВД МВД с Детства,0.0,0.0,1.0,0.0,"THE MONEY OF BELARUS, USA FOR TERRORISM IN LIFE TURCHENKO T.O. DOES NOT GIVE IN A DEBT TO BELARUS, SO THE KGB OF BELARUS WITH V.V. PUTIN WANT TO TAKE LUKASHENKO A.G. and HIS BAND of 6 PERSONS on the ZHIVTSA, on the Death of A.G. TURCHENKO, an innocent person, she was played as a Victim of the NKVD-MVD all my life from Childhood"


In [42]:
df_lukashenko.to_csv('df_lukashenko_translated.csv')